In [4]:
# --- CS-340 Module 7: Interactive Dashboard with Geolocation ---
from jupyter_dash import JupyterDash
from dash import dcc, html
import dash_leaflet as dl
from dash import dash_table
from dash.dependencies import Input, Output, State
import pandas as pd
import os
import base64
from animal_shelter import AnimalShelter  # Import CRUD module

# Setup MongoDB connection details
HOST = "nv-desktop-services.apporto.com"
PORT = int(os.getenv("MONGO_PORT", "32789"))  # Use today's session port
URI = f"mongodb://aacuser:SNHU1234@{HOST}:{PORT}/?authSource=admin&directConnection=true&retryWrites=false"

# Instantiate CRUD class with URI
shelter = AnimalShelter(uri=URI)

# Data Manipulation / Model
projection = {
    "age_upon_outcome": 1,
    "animal_id": 1,
    "animal_type": 1,
    "color": 1,
    "breed": 1,
    "date_of_birth": 1,
    "datetime": 1,
    "monthyear": 1,
    "outcome_subtype": 1,
    "name": 1,
    "outcome_type": 1,
    "sex_upon_outcome": 1,
    "age_upon_outcome_in_weeks": 1,
    "location_lat": 1,
    "location_long": 1,
    "location": 1,
    "_id": 0  # Drop _id so DataTable won't choke on ObjectId
}

# Fetch all records from the database
records = shelter.read({}, projection=projection, limit=None)

# Create a DataFrame from the fetched records
df = pd.DataFrame(records)

# Handle missing coordinates and clean data
df["location_lat"] = pd.to_numeric(df["location_lat"], errors="coerce")
df["location_long"] = pd.to_numeric(df["location_long"], errors="coerce")
df = df.dropna(subset=["location_lat", "location_long"]).reset_index(drop=True)

# Dashboard Layout / View
app = JupyterDash("CS340-Module7")

# Logo Import
logo_image = "/home/emmaliecole_snhu/Grazioso Salvare Logo.png"
encoded_image = base64.b64encode(open(logo_image, 'rb').read())

app.layout = html.Div([
    html.Div(id="hidden-div", style={"display": "none"}),

    # Logo Section
    html.Div(
        children=[
            html.Img(src=f'data:image/png;base64,{encoded_image.decode()}', 
                     style={"height": "150px", "width": "auto", "display": "block", "margin-left": "auto", "margin-right": "auto"}),
            html.Center(html.B(html.H1("Emmalie Cole — CS-340 Project 2 Dashboard"))),
        ],
        style={"text-align": "center"}
    ),

    html.Hr(),

    # Filter options (dropdown to filter by Animal Type)
    html.Div([
        html.Label("Filter by Animal Type:"),
        dcc.Dropdown(
            id='filter-dropdown',
            options=[
                {'label': 'Dog', 'value': 'Dog'},
                {'label': 'Cat', 'value': 'Cat'},
            ],
            value='Dog',  # default value
            multi=False,
            clearable=True,
            placeholder="Select Animal Type",
            style={'width': '50%'}
        ),
    ]),

    # Data Table Section
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": c, "id": c, "deletable": False, "selectable": True} for c in df.columns],
        data=df.to_dict('records'),
        page_size=10,
        sort_action="native",
        filter_action="native",
        row_selectable="single",
        selected_rows=[0],  # Pre-select the first row so the map renders immediately
        style_table={"overflowX": "auto", "height": "420px"},
        style_cell={"fontSize": 12, "padding": "6px"},
    ),

    html.Br(),
    html.Hr(),

    # Map Section
    html.Div(id="map-id", className="col s12 m6"),
])

# Interaction Between Components / Controller
@app.callback(
    Output("datatable-id", "data"),
    [Input("filter-dropdown", "value")]
)
def filter_data(animal_type):
    # Filter the DataFrame based on the selected animal type
    filtered_df = df[df['animal_type'] == animal_type]
    
    # Return the filtered data as a dictionary format for the DataTable
    return filtered_df.to_dict('records')


# Highlight selected column(s) in the DataTable
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    selected_columns = selected_columns or []
    return [{"if": {"column_id": c}, "backgroundColor": "#D2F3FF"} for c in selected_columns]


# Update the map for the selected row
@app.callback(
    Output("map-id", "children"),
    [Input("datatable-id", "derived_virtual_data"),
     Input("datatable-id", "selected_rows")]
)
def update_map(viewData, selected_rows):
    if not viewData:
        return html.Div("No data to display.")
    
    # Current view of the table
    dff = pd.DataFrame(viewData)
    row = selected_rows[0] if (selected_rows and len(selected_rows) > 0) else 0
    
    # Extract information for map display
    try:
        lat = float(dff.iloc[row]["location_lat"])
        lon = float(dff.iloc[row]["location_long"])
        breed = dff.iloc[row].get("breed", "Unknown breed")
        name = dff.iloc[row].get("name", "Unknown name")
    except Exception:
        return html.Div("Selected row has no coordinates.")
    
    # Return the Leaflet map with a marker for the selected animal
    return [
        dl.Map(
            style={"width": "100%", "height": "500px"},
            center=[30.75, -97.48],  # Austin, TX area
            zoom=10,
            children=[
                dl.TileLayer(id="base-layer-id"),
                dl.Marker(
                    position=[lat, lon],
                    children=[
                        dl.Tooltip(breed),
                        dl.Popup([html.H1("Animal Name"), html.P(name)]),
                    ],
                ),
            ],
        )
    ]

# Run the Dash app inside Jupyter
app.run_server(debug=True, mode="external", port=8050)

Dash app running on http://127.0.0.1:8050/
